# Authentication

In [ ]:
import json
import requests
import urllib3

urllib3.disable_warnings(category=urllib3.exceptions.InsecureRequestWarning)

# Use this if you are testing against an instance with OPENID auth
base_path= 'http://127.0.0.1:5002/api/v1'
username = "user1"
password = "user1"
%run fetch_openid_bearer.ipynb

# Setup requirements

For a simple test case, we need
* 1 DataSource
* The id of one of the configured llms (for chatting and ingestion)
* The id of one of the configured embedding models (for the ds)
* The id of one of the configured Vector Stores (for the ds)
* The id of one of the configured File Storages (for the ds)

In [ ]:
# Selecting an existing llm module
r = requests.get(f"{base_path}/modules/?type=LLM", verify=False, headers=auth_headers)
print("The following llm modules could be found in your system.")
if len(r.json()) == 0:
  print("You have no llm modules in your system. Please configure a llm module first and then continue by re-executing this cell.")
  exit(1)

for m_json in r.json():
    print(f"\t{m_json.get('name')} - ID: {m_json.get('id')}")

selected_llm_id = r.json()[0].get('id')

print(f"The llm with id {selected_llm_id} has been pre-selected for you. If you wish to take a different llm from the list above, please uncomment the following line and set the id accordingly")

In [ ]:
selected_llm_id = "51a9cf0c-f80d-4d1e-bff2-98b2bab9be3a"

In [ ]:
# Selecting an existing embedding model module
r = requests.get(f"{base_path}/modules/?type=EMBEDDING_MODEL", verify=False, headers=auth_headers)
print("The following embedding model modules could be found in your system.")
if len(r.json()) == 0:
  print("You have no embedding model modules in your system. Please configure one first and then continue by re-executing this cell.")
  exit(1)

for m_json in r.json():
    print(f"\t{m_json.get('name')} - ID: {m_json.get('id')}")

selected_embed_id = r.json()[0].get('id')

print(f"The embedding model with id {selected_embed_id} has been pre-selected for you. If you wish to take a different one from the list above, please uncomment the following line and set the id accordingly")

In [ ]:
selected_embed_id = '523ad0c0-e4cc-4d2c-8edf-65c148b52f97'

In [ ]:
# Selecting an existing vector store module
r = requests.get(f"{base_path}/modules/?type=VECTOR_STORE", verify=False, headers=auth_headers)
print("The following vector store modules could be found in your system.")
if len(r.json()) == 0:
  print("You have no vector store modules in your system. Please configure one first and then continue by re-executing this cell.")
  exit(1)

for m_json in r.json():
    print(f"\t{m_json.get('name')} - ID: {m_json.get('id')}")

selected_vector_store_id = r.json()[0].get('id')

print(f"The vector store with id {selected_vector_store_id} has been pre-selected for you. If you wish to take a different one from the list above, please uncomment the following line and set the id accordingly")

In [ ]:
selected_vector_store_id = 'd40144cf-10f9-4e5e-84ed-298a5e4a47f9'

In [ ]:
# Selecting an existing file storage module
r = requests.get(f"{base_path}/modules/?type=FILE_STORAGE", verify=False, headers=auth_headers)
print("The following file storage modules could be found in your system.")
if len(r.json()) == 0:
  print("You have no file storage modules in your system. Please configure one first and then continue by re-executing this cell.")
  exit(1)

for m_json in r.json():
    print(f"\t{m_json.get('name')} - ID: {m_json.get('id')}")

selected_file_storage_id = r.json()[0].get('id')

print(f"The file storage with id {selected_file_storage_id} has been pre-selected for you. If you wish to take a different one from the list above, please uncomment the following line and set the id accordingly")

In [ ]:
selected_file_storage_id = '28cb5c46-ff69-4b79-9399-24940a95b94a'

# Chat with the Workflow

First, we need a conversation that is linked to the newly created workflow.

In [ ]:
# Create DataSource
r = requests.post(f'{base_path}/data-source', headers=auth_headers, json={
  "name": "Notebook-Test-DS",
  "description": "Notebook-Test-DS Description",
  "filterTag": "Notebook-Test-DS-FilterTag",
  "embeddingModelId": selected_embed_id,
  "vectorStoreId": selected_vector_store_id,
  "fileStorageId": selected_file_storage_id
})
if r.status_code == 200:
  ds_id = r.json().get('id')
  print(f"A new Dats Source has been successfully created with ID '{ds_id}'.'")
else:
  print(r.status_code)
  print(r.text)
  exit(1)

In [ ]:
# Ingest text data
r = requests.post(f'{base_path}/ingest/text', headers=auth_headers, json={
  "llmId": selected_llm_id,
  "dataSourceId": ds_id,
  "overrides": [],
  "content": """
Homemade Bread Recipe (Classic White Bread)

Ingredients:
	•	4 cups (500g) bread flour (or all-purpose flour)
	•	1 ½ cups (360ml) warm water (100–110°F / 38–43°C)
	•	2 ¼ teaspoons (1 packet / 7g) active dry yeast or instant yeast
	•	2 tablespoons (30g) granulated sugar
	•	1 ½ teaspoons salt
	•	2 tablespoons (30g) unsalted butter, softened (or vegetable oil)

Step-by-Step Instructions:

Step 1: Activate the Yeast (If Using Active Dry Yeast)
	1.	Warm the water to about 100–110°F (38–43°C). If the water is too hot, it can kill the yeast; if it’s too cold, it won’t activate properly.
	2.	In a small bowl, combine the warm water, sugar, and yeast. Stir gently and let sit for 5–10 minutes until the mixture becomes foamy and bubbly. If it doesn’t foam, your yeast may be expired or the water was too hot/cold—start over with fresh yeast.
	3.	If using instant yeast, you can skip this activation step and mix it directly with the flour in the next step.

Step 2: Mix the Dough
	4.	In a large mixing bowl, combine the flour and salt. Mix well.
	5.	Add the softened butter (or oil) and mix into the flour.
	6.	Pour in the activated yeast mixture (or warm water if using instant yeast).
	7.	Using a wooden spoon (or a stand mixer with a dough hook), stir everything together until a shaggy dough forms.

Step 3: Knead the Dough
	8.	Lightly flour a clean work surface and turn the dough onto it.
	9.	Knead the dough by pressing it with the heel of your hand, folding it over itself, and rotating it slightly. Continue this motion for 8–10 minutes until the dough becomes smooth, elastic, and slightly tacky but no longer sticky.
	•	If the dough sticks too much, add a tablespoon of flour at a time. Avoid adding too much, or the bread may become dense.
	•	To check if it’s kneaded enough, use the windowpane test: Stretch a small piece of dough between your fingers. If it stretches thin enough to see light through without tearing, it’s ready.

Step 4: First Rise (Proofing)
	10.	Lightly grease a large bowl with oil or butter. Place the dough inside, turning it once to coat with oil.
	11.	Cover the bowl with a clean kitchen towel or plastic wrap. Let it rise in a warm, draft-free place for 1 to 2 hours, or until it doubles in size.

	•	If your kitchen is cold, you can place it inside a slightly warm (but turned off) oven or on top of a warm appliance.

Step 5: Shape the Dough
	12.	Once doubled in size, punch the dough down gently to release excess gas.
	13.	Turn it out onto a lightly floured surface and shape it into a rectangle.
	14.	Roll it into a tight log, tucking the edges under, to fit into a greased 9x5-inch loaf pan.
	15.	Cover loosely with a towel and let it rise again for 30–45 minutes, until it rises about 1 inch above the rim of the pan.

Step 6: Bake the Bread
	16.	Preheat your oven to 375°F (190°C) about 15 minutes before baking.
	17.	Place the loaf in the oven and bake for 25–30 minutes until golden brown.
	18.	To check doneness, tap the bottom of the loaf—it should sound hollow.
	19.	Alternatively, insert a thermometer into the center of the loaf. The internal temperature should be 190–200°F (88–93°C).

Step 7: Cool and Serve
	20.	Remove the bread from the oven and let it sit in the pan for 5 minutes.
	21.	Transfer it to a wire rack and let it cool completely before slicing—this helps keep the bread’s texture intact.
	22.	Once cooled, slice and enjoy!

Storage Tips:
	•	Store at room temperature in an airtight container for 2–3 days.
	•	For longer storage, wrap in plastic wrap and freeze for up to 3 months.
	•	To reheat, toast slices or warm them in an oven at 300°F (150°C) for a few minutes.
  """
})
if r.status_code == 200:
  print(f"Data has been ingested")
else:
  print(r.status_code)
  print(r.text)
  exit(1)

# Now chat with your workflow

In [ ]:
def stream_response(r):
    r.encoding = 'utf-8'
    if r.status_code == 200:
        for line in r.iter_lines():
            print(json.dumps(json.loads(line.decode('utf8')), indent=4))
        print()
    else:
        print(r.text)

In [ ]:
# Create Conversation
r = requests.post(f'{base_path}/conversation', headers=auth_headers, json={
  "name": "Notebook-Test-Conversation",
  "llmIds": [
    selected_llm_id
  ],
  "dataSourceIds": [
    ds_id
  ]
})
if r.status_code == 200:
  conversation_id = r.json().get('id')
  print(f"A new Conversation has been successfully created with ID '{conversation_id}'.'")
else:
  print(r.status_code)
  print(r.text)
  exit(1)

In [ ]:
r = requests.post(f'{base_path}/conversation/{conversation_id}/message', headers=auth_headers, json=
{
  "parts": [
    {
      "type": "TEXT",
      "content": "When I baked a bread, do you have some storage tips for me?"
    }
  ],
  "overrides": []
})
stream_response(r)